In [1]:
from bs4 import BeautifulSoup as bs
import regex
import os
from jinja2 import Environment, FileSystemLoader

In [2]:
with open('./pagina_resina.html', 'r', encoding='utf-8') as file:
    contenido_html_resina = file.read()

with open('./pagina_fdm.html', 'r', encoding='utf-8') as file:
    contenido_html_fdm = file.read()

pagina_resina = bs(contenido_html_resina, "html.parser")
pagina_fdm = bs(contenido_html_fdm, "html.parser")


In [3]:
marcas_resina = [marca.text.strip() for marca in pagina_resina.select_one("body > div > div.app.svelte-1hklul4 > main > div.flex.flex-col.md\:flex-row.p-6 > div.w-full.md\:w-1\/4 > div > div.mx-6 > div")]
marcas_fdm = [marca.text.strip() for marca in pagina_fdm.select_one("body > div > div.app.svelte-1hklul4 > main > div.flex.flex-col.md\:flex-row.p-6 > div.w-full.md\:w-1\/4 > div > div.mx-6 > div")]

<>:1: SyntaxWarning: invalid escape sequence '\:'
<>:2: SyntaxWarning: invalid escape sequence '\:'
<>:1: SyntaxWarning: invalid escape sequence '\:'
<>:2: SyntaxWarning: invalid escape sequence '\:'
/tmp/ipykernel_10357/874525791.py:1: SyntaxWarning: invalid escape sequence '\:'
  marcas_resina = [marca.text.strip() for marca in pagina_resina.select_one("body > div > div.app.svelte-1hklul4 > main > div.flex.flex-col.md\:flex-row.p-6 > div.w-full.md\:w-1\/4 > div > div.mx-6 > div")]
/tmp/ipykernel_10357/874525791.py:2: SyntaxWarning: invalid escape sequence '\:'
  marcas_fdm = [marca.text.strip() for marca in pagina_fdm.select_one("body > div > div.app.svelte-1hklul4 > main > div.flex.flex-col.md\:flex-row.p-6 > div.w-full.md\:w-1\/4 > div > div.mx-6 > div")]


In [4]:
productos_resina = pagina_resina.find("div", class_="w-full md:w-3/4 pl-0 md:pl-8").find_all("div", class_="flex flex-col sm:flex-row")
productos_fdm = pagina_fdm.find("div", class_="w-full md:w-3/4 pl-0 md:pl-8").find_all("div", class_="flex flex-col sm:flex-row")


In [5]:
print(len(productos_resina))
print(len(productos_fdm))
print(marcas_resina)
print(marcas_fdm)

31
114
['Creality', 'Elegoo', 'ANET', 'Anycubic', 'Formlabs', 'Phrozen']
['Bambu Lab', 'Prusa', 'Creality', 'AnkerMake', 'LulzBot', 'Kingroon', 'Artillery', 'FLSun', 'Sovol', 'Elegoo', 'Voxelab', 'ANET', 'Qidi', 'Anycubic', 'Modix', 'FlashForge', 'MakerBot', 'Snapmaker', 'Raise3D']


In [6]:
impresoras_resina = list()
impresoras_fdm = list()

for producto in productos_resina:
    nombre_producto = producto.find("h2", class_="text-lg font-bold mb-2").a.text.strip()
    marca = nombre_producto.split(" ")[0]
    subdiv = list(filter( lambda x: x != "" and x != " ", producto.find("div", class_="grid grid-cols-1 sm:grid-cols-2 gap-1 sm:gap-4").children))
    
    anio_lanzamiento = subdiv[1].text
    volumen_construccion = subdiv[3].text
    precio = subdiv[5].text.replace("$", "")
    moneda = "dolar"
    nombre_url = producto.find("img", class_="searchImage w-full sm:w-auto svelte-1mydttb").get("src").split("/")[2]
    url_imagen = f"https://ik.imagekit.io/3dpros/PrintersDB/{nombre_url}"

    impresoras_resina.append(
        {
            "nombre" : nombre_producto,
            "marca" : marca,
            "tipo" : "RESINA",
            "anio_lanzamiento" : anio_lanzamiento,
            "volumen_construccion" : volumen_construccion,
            "precio" : precio,
            "moneda" : "DOLAR",
            "url_imagen" : url_imagen
        }
    )

for producto in productos_fdm:
    nombre_producto = producto.find("h2", class_="text-lg font-bold mb-2").a.text.strip()
    marca = nombre_producto.split(" ")[0]
    subdiv = list(filter( lambda x: x != "" and x != " ", producto.find("div", class_="grid grid-cols-1 sm:grid-cols-2 gap-1 sm:gap-4").children))
    
    anio_lanzamiento = subdiv[1].text
    volumen_construccion = subdiv[3].text
    precio = subdiv[5].text.replace("$", "")
    moneda = "dolar"
    nombre_url = producto.find("img", class_="searchImage w-full sm:w-auto svelte-1mydttb").get("src").split("/")[2]
    url_imagen = f"https://ik.imagekit.io/3dpros/PrintersDB/{nombre_url}"

    impresoras_fdm.append(
        {
            "nombre" : nombre_producto,
            "marca" : marca,
            "tipo" : "FDM",
            "anio_lanzamiento" : anio_lanzamiento,
            "volumen_construccion" : volumen_construccion,
            "precio" : precio,
            "moneda" : "DOLAR",
            "url_imagen" : url_imagen
        }
    )

impresoras = impresoras_fdm + impresoras_resina


In [7]:
print(impresoras[0])

{'nombre': 'Bambu Lab P1S', 'marca': 'Bambu', 'tipo': 'FDM', 'anio_lanzamiento': '2023', 'volumen_construccion': '256 x 256 x 256 mm', 'precio': '699', 'moneda': 'DOLAR', 'url_imagen': 'https://ik.imagekit.io/3dpros/PrintersDB/Bambu_Lab_P1S'}


In [8]:
env = Environment(
    loader=FileSystemLoader('./plantilla_sql')  # Directorio donde están las plantillas SQL
)

template = env.get_template("script.sql")

sql_final = template.render(impresoras=impresoras)
with open("nuevo_script.sql", "w") as file:
    file.write(sql_final)

